In [2]:
from patchify import patchify
from PIL import Image
import cv2
import numpy as np
import splitfolders
import shutil

from prep_data import *

---

### Define helper function(s)

In [3]:
def move_files(source, destination):
    # gather all files
    allfiles = os.listdir(source)
    # iterate on all files to move them to destination folder
    for f in allfiles:
        src_path = os.path.join(source, f)
        dst_path = os.path.join(destination, f)
        shutil.move(src_path, dst_path)

---

### Patch large image into smaller pieces

In [4]:
patch_size = 1024
prep_images = PrepImages(
    image_directory_path='../data/images', 
    mask_directory_path='../data/masks',
    root_directory_path='../data/',
    patch_size=patch_size, 
    overlap = 0.2
)

In [5]:
remove_directories = ['n' + str(patch_size) + "_patches"]
for dir in remove_directories: 
    # print(dir)
    if dir in os.listdir(prep_images.root_directory_path): 
        shutil.rmtree(prep_images.root_directory_path + dir)
        print('Deleted images/masks in: ', prep_images.root_directory_path + dir)
    os.mkdir(prep_images.root_directory_path + dir)
    os.mkdir(prep_images.root_directory_path + dir + '/images')
    os.mkdir(prep_images.root_directory_path + dir + '/masks')
    os.mkdir(prep_images.root_directory_path + dir + '/images_with_useful_info')
    os.mkdir(prep_images.root_directory_path + dir + '/images_with_useful_info/masks')
    os.mkdir(prep_images.root_directory_path + dir + '/images_with_useful_info/images')

Deleted images/masks in:  ../data/n1024_patches


In [6]:
prep_images.create_patches()

../data/images/base_cropped.tif
Saved 128 images patches of 1024 x 1024 dimensions at: ../data/n1024_patches/images
../data/masks/base_mask.tif
Saved 136 masks patches of 1024 x 1024 dimensions at: ../data/n1024_patches/masks
../data/masks/base_mask_cropped.tif
Saved 128 masks patches of 1024 x 1024 dimensions at: ../data/n1024_patches/masks


---

### Remove images without sufficient class info

In [7]:
prep_images.remove_1class_images()

Total useful images are:  84
Total less-useful images are:  44


### Split up data into train/test folders 

In [8]:
input_folder = '../data/n' + str(patch_size) + '_patches/images_with_useful_info/'
output_folder = '../data/data_for_training_and_testing_' + 'n' + str(patch_size) + '/'
if os.path.exists(output_folder): 
    shutil.rmtree(output_folder)
splitfolders.ratio(input_folder, output=output_folder, seed=100, ratio=(.90, .10), group_prefix=None)

Copying files: 168 files [00:00, 351.79 files/s]


---

### Move data into correct directory structure for training

In [9]:
remove_directories = ['train_images', 'train_masks', 'val_images', 'val_masks']
for dir in remove_directories: 
    if dir in os.listdir(output_folder): 
        shutil.rmtree(output_folder + dir)
        print('Deleted files in: ', dir)
    os.mkdir(output_folder + dir)
    if dir in ['train_images', 'train_masks']: 
        os.mkdir(output_folder + dir + '/train')
    else: 
        os.mkdir(output_folder + dir + '/val')

In [10]:
move_files(output_folder + '/train/images/', output_folder + '/train_images/train/')
move_files(output_folder + '/train/masks/', output_folder + '/train_masks/train')
move_files(output_folder + '/val/images/', output_folder + '/val_images/val/')
move_files(output_folder + '/val/masks/', output_folder + '/val_masks/val')

In [11]:
shutil.rmtree(output_folder + '/train')
shutil.rmtree(output_folder + '/val')